# Epsim execution / plotting

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pickle
import itertools
from pathlib import Path
from gengraph import EpsimGraph
from epsim import Epsim
from read_building_csv import read_building_csv

In [ ]:
def get_cumulative(sim_result, prop, rnds):
    return sum([sim_result[rnd][prop] for rnd in rnds])


def write_csv(file_path, sim_result):
    with open(file_path, 'w') as f:
        f.write("round")
        for i in range(len(sim_result[0]['states'])):
            f.write(f",state_{i}")
        for key in sim_result[0].keys():
            if key != 'states':
                f.write(f",{key}")
        f.write("\n")

        for rnd, info in enumerate(sim_result):
            f.write(f"{rnd}")
            for num in info['states']:
                f.write(f",{num}")
            for key, val in info.items():
                if key != 'states':
                    f.write(f",{val}")
            f.write("\n")


def mean_of_runs(runs):    
    run_keys = runs[0][0].keys()

    # convert dict to list: remove keys
    rnd_data_per_run = list(zip(*runs))

    # compute mean over runs for every round statistic
    mean_per_rnd_data = [np.mean(list(zip(*[list(run.values())[1:] for run in rnd_data])), axis=1) \
                         for rnd_data in rnd_data_per_run]  # [1:]: remove states (treated seperatly)
    
    # add mean states
    mean_states_per_rnd_data = [np.mean(list(zip(*[list(run.values())[0] for run in rnd_data])), axis=1) \
                                for rnd_data in rnd_data_per_run]
    for rnd in range(len(mean_per_rnd_data)):
        l = list(mean_per_rnd_data[rnd])
        l.insert(0, mean_states_per_rnd_data[rnd])
        mean_per_rnd_data[rnd] = l

    # reconvert list to dict: readd keys
    mean_dict_per_rnd = [{key: rnd_data[i] for i, key in enumerate(run_keys)} for rnd_data in mean_per_rnd_data]
    return mean_dict_per_rnd


## Simple run
as an example of how to use the API

In [ ]:
# Simple run

starttime = time.time()
epsim_graph = EpsimGraph(k=100000, sigma_office=0.5, perc_split_classes=0.0, print_progress=True)
sim = Epsim(epsim_graph.family_nbrs, epsim_graph.school_nbrs_standard,
            epsim_graph.school_nbrs_split, epsim_graph.office_nbrs)
read_building_csv(sim, "input_data/salzburg_buildings.csv")
sim_result = sim.run_sim(sim_iters=200, num_start_agents=100, \
                         perc_immune_agents={'children': 0.21, 'parents': 0.36}, start_weekday=0, \
                         p_spread_family_dict={0: 0.9435}, p_spread_school_dict={0: 0.51}, p_spread_office_dict={0: 0.544}, p_detect_child_dict={0: 0.2}, \
                         p_detect_parent_dict={0: 0.5}, testing_dict={0: {'pcr': {'p': 0.95, 'weekdays': [2]}}, \
                         'antigen': {'p': 0.5, 'weekdays': [0, 4]}}, omicron=True, split_stay_home=False, \
                         print_progress=True)
print(f"runtime: {time.time() - starttime}s")

In [ ]:
# print sim statistics
print(f"total infected: {get_cumulative(sim_result, 'infected', range(len(sim_result)))}")
print(f"total infected by children: {get_cumulative(sim_result, 'infected_by_children', range(len(sim_result)))}")
print(f"total infected children: {get_cumulative(sim_result, 'infected_children', range(len(sim_result)))}")
if len(sim_result) > 50:
    print(f"infected first 50 rounds: {get_cumulative(sim_result, 'infected', range(50))}")
    print(f"infected by children first 50 rounds: {get_cumulative(sim_result, 'infected_by_children', range(50))}")
    print(f"infected children first 50 rounds: {get_cumulative(sim_result, 'infected_children', range(50))}")

## Multiple runs per parameter combination
for plotting an averaged diagram of multiple runs to avoid outliners

In [ ]:
# Define simulation parameters
sim_iters = 400

def gen_graph_run_sim(k, sigma_office, num_start_agents, perc_immune_agents, start_weekday, perc_split_classes,
                      p_spread_family_dict, p_spread_school_dict, p_spread_office_dict, p_detect_child_dict, p_detect_parent_dict, testing_dict,
                      omicron, split_stay_home):
    epsim_graph = EpsimGraph(k, sigma_office, perc_split_classes)
    sim = Epsim(epsim_graph.family_nbrs, epsim_graph.school_nbrs_standard, epsim_graph.school_nbrs_split,
                epsim_graph.office_nbrs)
    read_building_csv(sim, "input_data/salzburg_buildings.csv")
    return sim.run_sim(sim_iters, num_start_agents, perc_immune_agents, start_weekday, p_spread_family_dict,
                       p_spread_school_dict, p_spread_office_dict, p_detect_child_dict, p_detect_parent_dict, testing_dict, 
                       omicron, split_stay_home)


sim_params = {
    'k': [100000],
    'sigma_office': [0.5],
    'num_start_agents': [100],
    'perc_immune_agents': [tuple({'children': 0.21, 'parents': 0.36}.items())],
    'start_weekday': [0],
    'perc_split_classes': [0.0],
    'p_spread_family_dict': [tuple({0: 0.9435}.items())],
    'p_spread_school_dict': [tuple({0: 0.51}.items())],
    'p_spread_office_dict': [tuple({0: 0.544}.items())],
    'p_detect_child_dict': [tuple({0: 0.2}.items())],
    'p_detect_parent_dict': [tuple({0: 0.5}.items())],
    'testing_dict': [tuple({0: tuple({'pcr': tuple({'p': 0.95, 'weekdays': (2,)}.items()), \
                    'antigen': tuple({'p': 0.5, 'weekdays': (0, 4)}.items())}.items())}.items())],
    'omicron': [True],
    'split_stay_home': [False]
}

param_combis = list(itertools.product(*sim_params.values()))

In [ ]:
# Execute simulations: multiple runs per parameter combination

num_runs = 1
runs_per_param_combi = {pc: [gen_graph_run_sim(*pc) for i in range(num_runs)] for pc in param_combis}
pickle.dump(runs_per_param_combi, open("runs_per_param_combi.p", 'wb'))

In [ ]:
# Plot mean infections per round over all runs of one parameter combination to avoid outliners

plt.rcParams.update({'font.size': 15})

for pc, runs in runs_per_param_combi.items():
    params = dict(zip(sim_params, pc))
    print(params)
    
    mean_dict_per_rnd = mean_of_runs(runs)
    
    mean_infecs_per_rnd = [sum(rnd_dict['states'][1:]) for rnd_dict in mean_dict_per_rnd]
    total_infected = get_cumulative(mean_dict_per_rnd, 'infected', range(len(mean_dict_per_rnd)))
    total_infected_by_children = get_cumulative(mean_dict_per_rnd, 'infected_by_children', \
                                                range(len(mean_dict_per_rnd)))
    total_infected_children = get_cumulative(mean_dict_per_rnd, 'infected_children', range(len(mean_dict_per_rnd)))
    average_infected_by_children = total_infected_by_children / total_infected_children
    infected_by_children_first_50 = get_cumulative(mean_dict_per_rnd, 'infected_by_children', range(50))
    infected_children_first_50 = get_cumulative(mean_dict_per_rnd, 'infected_children', range(50))
    average_infected_by_children_first_50 = infected_by_children_first_50 / infected_children_first_50
    
    fig, axs = plt.subplots(figsize=(20,13))
    fig.patch.set_facecolor('xkcd:white')
    axs.set_ylim(top=238640)
    axs.set_xlim(0, 200)
    axs.plot(range(len(mean_infecs_per_rnd)), mean_infecs_per_rnd)
    plt.annotate(238640, xy=(-0.065, 238640), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points')
    plt.annotate(int(max(mean_infecs_per_rnd)), xy=(1, int(max(mean_infecs_per_rnd))), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points')
    plt.annotate(int(min(mean_infecs_per_rnd)), xy=(0, int(min(mean_infecs_per_rnd))), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points')
    axs.set_xlabel("Rounds")
    axs.set_ylabel(f"Infected agents (mean of {num_runs} runs)")

    axs.set_title(f"Omicron Variant (p_spread_school={params['p_spread_school_dict'][0]}, " \
                  + f"p_spread_office={params['p_spread_office_dict'][0]})")
    text_params = f"k={params['k']},  " \
                 + f"sigma_office={params['sigma_office']},  " \
                 + f"num_start_agents={params['num_start_agents']},  " \
                 + f"perc_immune_agents={params['perc_immune_agents']},  " \
                 + f"start_weekday={params['start_weekday']},\n" \
                 + f"perc_split_classes={params['perc_split_classes']},  " \
                 + f"p_spread_family_dict={params['p_spread_family_dict']},  " \
                 + f"p_spread_school_dict={params['p_spread_school_dict']},  " \
                 + f"p_spread_office_dict={params['p_spread_office_dict']},  " \
                 + f"p_detect_child_dict={params['p_detect_child_dict']},  " \
                 + f"p_detect_parent_dict={params['p_detect_parent_dict']},\n" \
                 + f"testing_dict={params['testing_dict']},  " \
                 + f"omicron={params['omicron']},  " \
                 + f"split_stay_home={params['split_stay_home']}"
    axs.text(0, -0.14, text_params, transform=axs.transAxes, wrap=True)
    text_stats = f"average persons infected by children: {average_infected_by_children}\n" \
                + f"average persons infected by children first 50 rounds: {average_infected_by_children_first_50}\n"
    axs.text(0, -0.24, text_stats, transform=axs.transAxes, wrap=True)

    plt.show()

In [ ]:
# Write csv of mean values
for pc, runs in runs_per_param_combi.items():
    params = dict(zip(sim_params, pc))
    path = Path(f"csv/p_spread_school_{params['p_spread_school_dict'][0]}_p_spread_office_{params['p_spread_office_dict'][0]}.csv")
    write_csv(path, mean_of_runs(runs))

In [ ]:
# Plot histogram of total infections for all runs of one parameter combination
# This shows how much the total infections of different simulations with the same parameters deviatie due to
# the randomness of the simulation

plt.rcParams.update({'font.size': 12})

for pc, runs in runs_per_param_combi.items():
    print(dict(zip(sim_params, pc)))
    total_infec_per_run = [sum(run[-1]['states'][1:]) for run in runs]
    fig, axs = plt.subplots(figsize=(10,6))
    fig.patch.set_facecolor('xkcd:white')
    axs.set_xlabel("Total infected agents")
    axs.set_ylabel("# Runs")
    axs.hist(total_infec_per_run)
    plt.show()
    print()